In [1]:
import pandas as pd
import seaborn as sns
import glob
from numpy import genfromtxt
# from sklearn.metrics import f1_score
from matplotlib import pyplot as plt
import numpy as np
# https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html
import json
from collections import OrderedDict
import os
import re
import logging
import multiprocessing
from functools import partial
from datetime import datetime
from Bio import SeqIO
from Bio.Seq import Seq
import gzip
from sklearn import preprocessing
from sklearn.feature_selection import VarianceThreshold

/data/hoan/mybin/miniconda3/envs/py36/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Create map from gene ID to cluster ID

In [2]:
# panta input directory
# pantain_dir = '/data/hoan/amromics/prediction/data/Ecoli1936/prokkatest/'
pantain_dir = '/data/hoan/amromics/prediction/data/Ecoli1936/prokka/'
# panta output dir
# pantaout_dir = '/data/hoan/amromics/prediction/output/pantaEcoli1936aligntest/'
pantaout_dir = '/data/hoan/amromics/prediction/output/pantaEcoli1936align_v4/'

In [3]:
with open(pantaout_dir + 'annotated_clusters.json', 'r') as JSON:
    json_dict = json.load(JSON)
# data = json.loads('/data/hoan/amromics/prediction/output/pantaEcoli1936aligntest/clusters.json')[0]

In [4]:
# json_dict

In [5]:
gene2clusterdict = {}
for key in json_dict:
    if len(json_dict[key])==0:
        gene2clusterdict[key] = key
    for gene in json_dict[key]['gene_id']:
        gene2clusterdict[gene] = key

### Find all AMR genes

In [6]:
def parse_gff_AMRgene_finder(gff_fh, sample_id, min_protein_len=40):
    # gene_annotation = OrderedDict()
    # gene_position = OrderedDict()    
    # suffix = 1
    # bed_records = []
    # gene_index = 0
    seq_id = None
    min_cds_len = 3 * min_protein_len
    gene_list = []
    
    for line in gff_fh:            
        if line.startswith('##FASTA'):
            #Done reading gff, move on to reading fasta
            break

        if line[0] == '#':
            continue
        line = line.strip()
        #print(line)
        cells = line.split('\t')
        if cells[2] != 'CDS':
            continue
        if 'BARRGD' not in cells[8]:
            continue
        start = int(cells[3])
        end = int(cells[4])
        length = end - start + 1
        if length < min_cds_len:
            continue
        if length % 3 != 0:
            continue
        cells[0] = cells[0].replace('-','_') #make sure seq_id has no -
        
        if seq_id != cells[0]:
            seq_id = cells[0]
            gene_index = 0

        # strand = cells[6]
        tags = cells[8].split(';')
        gene_id = None
        gene_name = ''
        gene_product = ''
        for tag in tags:
            if tag.startswith('ID='):
                gene_id = tag[3:]
            elif tag.startswith('gene='):                    
                gene_name = tag[5:]
                gene_name = re.sub(r'\W', '_', gene_name)
            elif tag.startswith('product='):                    
                gene_product = tag[8:]
        if gene_id == None:
            continue

        # Ensure gene_id is in the format of sample_id-seq_id-gene_tag
        if not gene_id.startswith(sample_id + '-'):
            gene_id = sample_id + '-' + gene_id

        if not gene_id.startswith(sample_id + '-' + seq_id + '-'):
            gene_id = sample_id + '-' + seq_id + '-' + gene_id[len(sample_id)+1:]

        gene_list.append(gene_id)
    
    return gene_list

In [7]:
amr_gene = []
for data_dir in glob.glob(pantain_dir + '*.gff'):
    # print(data_dir)
    in_fh = open(data_dir)
    sample_id = data_dir.split('/')[-1][:-4]
    amr_gene += parse_gff_AMRgene_finder(in_fh, sample_id)
    in_fh.close()

In [8]:
amr_gene[:3], len(amr_gene)

(['SAMEA2204230.contig-SAMEA2204230.contig00001-KJJADFBE_00063',
  'SAMEA2204230.contig-SAMEA2204230.contig00001-KJJADFBE_00095',
  'SAMEA2204230.contig-SAMEA2204230.contig00001-KJJADFBE_00151'],
 119509)

## TODO: Map genes back to cluster IDs

In [9]:
amr_clusterID = [gene2clusterdict[gene] for gene in amr_gene]
amr_clusterID = list(set(amr_clusterID))

In [10]:
amr_clusterID[:3]

['aacA16', 'fosA7', 'aadA1_10624']

In [11]:
len(amr_clusterID)

464

### Common cluster from matrix

In [12]:
pa_matrix = pd.read_csv(pantaout_dir+'gene_presence_absence.Rtab', sep='\t', index_col=0).T

In [13]:
pa_matrix.head(2)

Gene,groups_0,namA,groups_2,groups_3,groups_4,groups_5,groups_6,groups_7,groups_8,groups_9,...,groups_74779,groups_74780,groups_74781,groups_74782,traI_2_16929,groups_74784,groups_74785,groups_74786,groups_74787,groups_74788
SAMEA2204229.contig,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SAMEA2204230.contig,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
n_samples = pa_matrix.shape[0]
n_genes = pa_matrix.shape[1]
n_genes

74789

In [15]:
colsum = pa_matrix.sum()

In [16]:
common_gene_cluster = [colsum.index[idx] for idx in range(n_genes) if colsum[idx] > 0.99*n_samples]

In [17]:
common_gene_cluster[:4], len(common_gene_cluster)

(['rsmI', 'ssb_1', 'rpnA', 'rluF'], 2114)

### Read prepresentative sequence

In [18]:
from Bio import SeqIO
genecluster2representativeseq = {}
with open(pantaout_dir+'representative_clusters_prot.fasta') as handle:
    for record in SeqIO.parse(handle, "fasta"):
        name, sequence = record.id, str(record.seq)
        genecluster2representativeseq[name] = sequence
        # print(name,'----', sequence)

### Compute label encoder for gene cluster

In [19]:
with open(pantaout_dir + 'samples.json', 'r') as JSON:
    sample_dict = json.load(JSON)
sample2integerindex = {}
for idx in range(len(sample_dict)):
    sample2integerindex[sample_dict[idx]['id']] = idx
n_samples = len(sample_dict)

In [20]:
# computed_gene_cluster = common_gene_cluster; #amr_clusterID
computed_gene_cluster = amr_clusterID;

In [21]:
# amr_mat = None;
ksize = 10;
kmer_list = [];
amr_mat = np.zeros((n_samples, len(computed_gene_cluster)))
pairdata = []
for idx in range(len(computed_gene_cluster)):
    alignment_dir = pantaout_dir + 'clusters/' + computed_gene_cluster[idx] +'/'+computed_gene_cluster[idx]+'.faa.aln.gz'
    # alignment_dir = pantaout_dir + 'clusters/' + computed_gene_cluster[idx] +'/'+computed_gene_cluster[idx]+'.fna.aln.gz'
    with gzip.open(alignment_dir, "rt") as handle:
        for record in SeqIO.parse(handle, "fasta"):
            name, sequence = record.id, str(record.seq)
            sample_id = name.split('-')[0]
            seqraw = sequence.replace('-', '')
            n_kmers = len(seqraw) - ksize + 1
            for i in range(n_kmers):
                kmer = seqraw[i:i + ksize]
                kmer_list.append(kmer)
                pairdata.append((sample2integerindex[sample_id], kmer))

LNYSICDIAESNELILEAAKMLKKSFLDAGNESWGDIKNAIEEVEECIEHPNICLGICLDDKLIGWTGLRPMYDKTWELHPMVIKTEYQGKGFGKVLLRELETRAKSRGIIGIALGTDDEYQKTSLSMIDINERNIFDEIGNIKNVNNHPYEFYKKCGYMIVGIIPNANGKRKPDIWMWKDIS
LNYSICDIAESNELILEAAKMLKKSFLDAGNESWGDIKNAIEEVEECIEHPNICLGICLDDKLIGWTGLRPMYDKTWELHPMVIKTEYQGKGFGKVLLRELETRAKSRGIIGIALGTDDEYQKTSLSMIDINERNIFDEIGNIKNVNNHPYEFYKKCGYMIVGIIPNANGKRKPDIWMWKDIS


In [22]:
unique_kmer = list(set(kmer_list))

In [23]:
# AMR genes: (74198447, 169115)
len(kmer_list), len(unique_kmer)

(74198447, 169115)

In [24]:
kmer2index = {}
for i in range(len(unique_kmer)):
    kmer2index[unique_kmer[i]] = i

In [25]:
kmer_matrix = np.zeros((n_samples, len(unique_kmer)))

In [26]:
# ct = 0
for u, v in pairdata:
    kmer_matrix[u, kmer2index[v]] = 1

In [27]:
# np.save(pantaout_dir + 'kmer_amr_mat.npy', kmer_matrix) # save numpy array

In [28]:
kmer_matrix.shape

(1653, 169115)

In [29]:
selector = VarianceThreshold(threshold=0.01)
kmer_matrix_VT = selector.fit_transform(kmer_matrix)

In [30]:
kmer_matrix_VT.shape

(1653, 58689)

In [31]:
# np.save(pantaout_dir + 'kmer_amr_mat_VT1.npy', kmer_matrix_VT) # save numpy array
# np.save(pantaout_dir + 'kmer_coregene_mat_VT1.npy', kmer_matrix_VT) # save numpy array

In [32]:
stop

NameError: name 'stop' is not defined

In [ ]:
def count_differ_char(seq1, seq2):
    count = 0
    for i in range(len(seq1)):
        if seq1[i] != seq2[i]:
            count += 1
    return(count)

In [ ]:
def count_similar_char(seq1, seq2):
    count = 0
    for i in range(len(seq1)):
        if seq1[i] == seq2[i]:
            count += 1
    return(count)

In [ ]:
# count_differ_char('DovanHoan', 'DoVANHOAN')

In [ ]:
# amr_mat = None;
amr_mat = np.zeros((n_samples, len(computed_gene_cluster)))
for idx in range(len(computed_gene_cluster)):
    alignment_dir = pantaout_dir + 'clusters/' + computed_gene_cluster[idx] +'/'+computed_gene_cluster[idx]+'.faa.aln.gz'
    # mat = None
    # index = 0
    # index_set = []
    # print(idx, end = ',')
    with gzip.open(alignment_dir, "rt") as handle:
        for record in SeqIO.parse(handle, "fasta"):
            name, sequence = record.id, str(record.seq)
            sample_id = name.split('-')[0]
            amr_mat[sample2integerindex[sample_id], idx] = count_similar_char(genecluster2representativeseq[computed_gene_cluster[idx]], sequence)
            # index_set.append(sample2integerindex[sample_id])
            # print(record.id)
            # print(genecluster2representativeseq[computed_gene_cluster[idx]],':::::', sequence)

In [ ]:
# amrgene_annotation = pd.DataFrame({'gene': amr_clusterID, 'start_index': start_idx, 'end_index': end_idx})

In [ ]:
# amrgene_annotation.head(2)

In [ ]:
# amrgene_annotation.to_csv(pantaout_dir + 'amrgene_annotation_VarianceThreshold.csv', index=None)
# amrgene_annotation.to_csv(pantaout_dir + 'amrgene_annotation.csv', index=None)

In [ ]:
# pd.DataFrame(amr_mat).to_csv(pantaout_dir + 'amrlabelencodermat_VarianceThreshold.csv', index=None)

In [ ]:
# amrtest = pd.read_csv(pantaout_dir + 'amrlabelencodermat.csv')
# np.save(pantaout_dir + 'amrlabelencodermat.npy', amrtest.values) # save numpy array

In [ ]:
# amrtest.values[:3,:3]

In [ ]:
# Full matrix shape: (1653, 174005)
# amr_mat.shape

In [ ]:
# np.save(pantaout_dir + 'amrlabelencodermat_VarianceThreshold.npy', amr_mat) # save numpy array
np.save(pantaout_dir + 'similarsitecolsum1pct.npy', amr_mat) # save numpy array
# np.save(pantaout_dir + 'amrlabelencodermat.npy', amr_mat) # save numpy array

In [ ]:
stop

In [ ]:
# load data
amr_mat = np.load(pantaout_dir + 'amrlabelencodermat_VT10.npy')

In [ ]:
amr_mat.shape, np.max(amr_mat[:50,:3000])

In [ ]:
def one_hot_encode(seq):
    mapping = dict(zip("ACGT", range(4)))    
    seq2 = [mapping[i] for i in seq]
    return np.eye(4)[seq2]

one_hot_encode("AACGT")

In [ ]:
mapping = dict()
for i in range(22):
    mapping[i] = i
def one_hot_encode2(seq):
    seq2 = [mapping[i] for i in seq]
    return np.eye(22)[seq2].flatten()

# one_hot_encode([1,2,3])

In [ ]:
amr_mat = amr_mat.astype(int)
amr_matOnehot = None
for idx in range(amr_mat.shape[0]):
    if idx == 0:
        amr_matOnehot = one_hot_encode2(amr_mat[idx,:])
    else:
        amr_matOnehot = np.vstack([amr_matOnehot, one_hot_encode2(amr_mat[idx,:])])

In [ ]:
amr_matOnehot.shape, amr_mat.shape

In [ ]:
amr_matOnehot